# L1: Word representations

In this lab you will implement the **skip-gram model with negative sampling (SGNS)** from Lecture&nbsp;1.4, and use it to train word embeddings on the text of the [Simple English Wikipedia](https://simple.wikipedia.org/wiki/Main_Page).

⚠️ The dataset for this lab contains 18M tokens. This is very little as far as word embedding datasets are concerned – for example, the original word2vec model was pre-trained on 100B tokens. In spite of this, you will need to think about efficiency when processing the data and training your models. In particular, wherever possible you should use iterators rather than lists, and vectorize operations (using [NumPy](https://numpy.org) or [PyTorch](https://pytorch.org)) as much as possible.

## Load the data

The data for this lab comes as a bz2-compressed plain text file. It consists of 1,163,769 sentences, with one sentence per line and tokens separated by spaces. The cell below contains a wrapper class `SimpleWikiDataset` that can be used to iterate over the sentences (lines) in the text file. On the Python side of things, each sentence is represented as a list of tokens (strings).

In [1]:
import bz2

class SimpleWikiDataset():
    
    def __init__(self, max_sentences=None):
        self.max_sentences = max_sentences
    
    def __iter__(self):
        with bz2.open('simplewiki.txt.bz2', 'rt') as sentences:
            for i, sentence in enumerate(sentences):
                if self.max_sentences and i >= self.max_sentences:
                    break
                yield sentence.split()

Using this class, we define two variants of the dataset: the full dataset and a minimal version with the first 1% of the sentences in the full dataset. The latter will be useful to test code without running it on the full dataset.

In [2]:
# Dataset with all sentences (N = 1,163,769)
full_dataset = SimpleWikiDataset()

# Minimal dataset
mini_dataset = SimpleWikiDataset(max_sentences=11638)

The next code cell defines a generator function that allows you to iterate over all tokens in a dataset:

In [3]:
def tokens(sentences):
    for sentence in sentences:
        for token in sentence:
            yield token

To illustrate how to use this function, here is code that prints the number of tokens in the full dataset:

In [4]:
print(sum(1 for t in tokens(full_dataset)))

17594885


## Problem 1: Build the vocabulary and frequency table

Your first task is to construct the embedding **vocabulary** – the set of unique words that will receive an embedding. Because you will eventually need to map words to vector dimensions, you will represent the vocabulary as a dictionary that maps words (strings) to a contiguous range of integers.

Along with the vocabulary, you will also construct the **frequency table**, that is, the table that holds the absolute frequencies (counts) in the data, for all words in your vocabulary. This will simply be an array of integers, indexed by the word ids in the vocabulary.

To construct the vocabulary and the frequency table, complete the skeleton code in the cell below:

In [7]:
import numpy as np
from collections import defaultdict
def make_vocab_and_counts(sentences, min_count=5):
     # TODO: Replace the next line with your own code\n",
     count = defaultdict(int)
     vocab = {}
     for token in tokens(sentences):
         #print(token)
         count[token] += 1
         if token not in vocab:
         #print (len(vocab))
             vocab[token] = len(vocab)
     count_2 = list(count.values())
     vocab_2 = vocab.copy()
     counter = 0
     for key, value in vocab.items():
         if list(count.values())[counter] < min_count:
             vocab_2.pop(key, 0)
             count_2[counter] = 0
         counter = counter + 1
     #count_2 = [i for i in count_2 if i != 0]
     return vocab_2, count_2
 #return {}, np.array(())

Your code should comply with the following specification:

**make_vocab_and_counts** (*sentences*, *min_count* = 5)

> Reads from an iterable of *sentences* (lists of string tokens) and returns a pair *vocab*, *counts* where *vocab* is a dictionary representing the vocabulary and *counts* is a 1D-[ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) with the absolute frequencies (counts) of the words in the vocabulary. The dictionary *vocab* maps words to a contiguous range of integers starting at&nbsp;0. In the *counts* array, the entry at index $i$ is the count of that word in *vocab* which maps to $i$. Words that occur less than *min_count* times are excluded from the vocabulary.

### 🤞 Test your code

To test your code, print the sizes of the vocabularies constructed from the two datasets, as well as the count totals. The correct vocabulary size for the minimal dataset is 3,231; for the full dataset, the correct vocabulary size is 73,339. The correct totals are 155,818 for the minimal dataset and 17,297,355 for the full dataset.

In [8]:
mini_dataset = SimpleWikiDataset(max_sentences=11638)
vocab, counts = make_vocab_and_counts (mini_dataset, min_count = 5)
print (vocab)
print (counts)
print (len(vocab))
print (sum(counts))
#mini_dataset_preprocess= preprocess(vocab, counts, mini_dataset, threshold=0.001)
#vocab_p, counts_p = make_vocab_and_counts (mini_dataset_preprocess, min_count = 5)
#print (len(vocab_p))
#print (sum(counts_p))

{'their': 0, 'name': 1, 'is': 2, 'usually': 3, 'said': 4, 'as': 5, 'it': 7, 'can': 8, 'not': 9, 'be': 10, 'searched': 11, 'for': 12, 'on': 13, 'written': 16, 'like': 17, 'that': 18, 'they': 19, 'started': 20, 'in': 21, 'california': 23, 'have': 24, 'released': 25, 'five': 26, 'albums': 27, 'fifth': 28, 'album': 29, 'was': 30, 'april': 31, 'gold': 32, 'standard': 33, 'or': 36, 'how': 37, 'i': 38, 'to': 40, 'stop': 41, 'and': 43, 'love': 44, 'the': 47, 'episode': 48, 'of': 49, 'season': 50, 'first': 52, 'shown': 53, 'television': 54, 'december': 55, 'springfield': 56, 'its': 59, 'economy': 60, 'burns': 61, 'a': 63, 'where': 65, 'bart': 73, 'his': 74, 'appearance': 76, 'state': 78, 'howard': 81, 'also': 85, 'known': 86, 'yellow': 87, 'spotted': 88, 'tree': 89, 'frog': 90, 'new': 91, 'england': 92, 'bell': 95, 'from': 96, 'eastern': 97, 'australia': 98, 'people': 99, 'thought': 100, 'these': 101, 'frogs': 102, 'were': 103, 'but': 105, 'then': 106, 'scientists': 107, 'found': 108, 'some': 1

## Problem 2: Preprocess the data

Your next task is to preprocess the training data. This involves the following:

* Discard words that are not in the vocabulary
* Map each word to its vocabulary id
* Randomly discard words according to the subsampling strategy covered in Lecture&nbsp;1.4
* Discard sentences that have become empty

As a reminder, the subsampling strategy involves discarding tokens $w$ with probability

$$
P(w) = \max (0, 1-\sqrt{tN/\#(w)})
$$

where $\#(w)$ is the count of $w$, $N$ is the total number of counts, and $t$ is the chosen threshold (default value: 0.001).

The cell below contains skeleton code for a generator function `preprocess`:

In [9]:
def preprocess(vocab, counts, sentences, threshold=0.001):
     # TODO: Replace the next line with your own code
     #Discard words that are not in the vocabulary, for a sentence
    data = sentences
    vocab_keys= list
    total_counts = sum(counts)
    preprocessed_list = []
    for sentence in data:
        temp_list = []
        for token in sentence:
            if token not in vocab:
                 sentence = [x for x in sentence if x != token]
            else:
                prob = 1 - (np.sqrt((threshold*total_counts)/counts[vocab[token]]))
                if prob < 0:
                     prob = 0
         #if w == N -> p == 0.97
                if (np.random.random()) > prob:
                    temp_list.append(vocab[token])
        if temp_list:
            #print (temp_list)
            #preprocessed_list.append(temp_list)
            yield temp_list
    #print(preprocessed_list)
    #yield preprocessed_list

Extend this skeleton code into a function that implements the preprocessing. Your code should comply with the following specification:

**preprocess** (*vocab*, *counts*, *sentences*, *threshold* = 0.001)

> Reads from an iterable of *sentences* (lists of string tokens) and yields the preprocessed sentences as non-empty lists of word ids (integers). Words not in *vocab* are discarded. The remaining words are randomly discarded according to the subsampling strategy with the given *threshold*. In the non-empty sentences, each token is replaced by its id in the vocabulary.

**⚠️ Please observe** that your function should *yield* the preprocessed sentences, not return a list with all of them. That is, we ask you to write a *generator function*. If you have not worked with generators and iterators before, now is a good time to read up on them. [More information about generators](https://wiki.python.org/moin/Generators)

### 🤞 Test your code

Test your code by comparing the total number of tokens in the preprocessed version of each dataset with the corresponding number for the original data. The former should be ca. 59% of the latter for the minimal dataset, and ca. 69% for the full dataset. The exact percentage will vary slightly because of the randomness in the sampling. You may want to repeat your computation several times.

In [11]:
# TODO: Test your code here
# TODO: Test your code here
#11638
mini_dataset = SimpleWikiDataset(max_sentences=11638)
#full_dataset = SimpleWikiDataset()
vocab, counts = make_vocab_and_counts (mini_dataset, min_count = 5)
print (len(vocab))
print (len(counts))
#print (counts[0])
print (sum(counts))
#print (counts)
#print (vocab)
mini_dataset_preprocess= preprocess(vocab, counts, mini_dataset, threshold=0.001)
#print(mini_dataset_preprocess)
vocab_p, counts_p = make_vocab_and_counts (mini_dataset_preprocess, min_count = 5)
print (len(vocab_p))
print (sum(counts_p))

3231
12437
155818
3231
101496


## Problem 3: Generate the training examples

Your next task is to translate the preprocessed sentences into training examples for the skip-gram model: both *positive examples* (target word–context word pairs actually observed in the data) and *negative examples* (pairs randomly sampled from a noise distribution).

**⚠️ We expect that solving this problem will take you the longest time in this lab.**

### General strategy

The general plan for solving this problem is to implement a generator function that traverses the preprocessed sentences, at each position of the text samples a window, and then extracts all positive examples from it. For each positive example, the function also generates $k$ negative examples, where $k$ is a hyperparameter. Finally, all examples (positive and negative) are combined into the tensor representation described below.

### Representation

How should you represent a batch of training examples? Writing $B$ for the batch size, the obvious choice would be to represent the inputs as a matrix of shape $[B, 2]$ and the output labels (positive/negative) as a vector of length $B$. This representation would be quite wasteful on the input side, however, as each target word (index) from a positive example would have to be repeated in all negative samples. For example ($k=3$):

Here you will use a different representation: First, instead of a single input batch, there will be a *pair* of input batches – a vector for the target words and a matrix for the context words. If the target word vector has length $B$, the context word matrix has shape $[B, 1+k]$. The $i$th element of the target word vector is the target word for *all* context words in the $i$th row of the context word matrix: the first column of that row comes from a positive example, the remaining columns come from the $k$ negative samples. Accordingly, the batch with the output labels will be a matrix of the same shape as the context word matrix, with its first column set to&nbsp;1 and its remaining columns set to&nbsp;0. Corresponding to the example above:

For the present problem, you will only be concerned with the two input batches; the output batch will be constructed in the training procedure. In fact, for a fixed batch size $B$, that batch is always exactly the same, so you will only have to build it once.

### Negative sampling

Recall from Lecture&nbsp;1.4 that the probability of a word $c$ to be selected as the context word in a negative sample is proportional to its exponentiated count $\#(c)^\alpha$, where $\alpha$ is a hyperparameter (default value: 0.75).

To implement negative sampling from this distribution, you can follow a standard recipe: Start by pre-computing an array containing the *cumulative sums* of the exponentiated counts. Then, generate a random cumulative count $n$, and find that index in the pre-computed array at which $n$ should be inserted to keep the array sorted. That index identifies the sampled context word.

All operations in this recipe can be implemented efficiently in PyTorch; the relevant functions are [`torch.cumsum`](https://pytorch.org/docs/stable/generated/torch.cumsum.html) and [`torch.searchsorted`](https://pytorch.org/docs/stable/generated/torch.searchsorted.html). For optimal efficiency, you should sample all $B \times k$ negative examples in a batch at once.

Here is skeleton code for this problem:

In [74]:
def training_examples(vocab, counts, sentences, window=5, num_ns=5, batch_size=1<<19, ns_exponent=0.75):
    # TODO: Replace the next line with your own code
    target_words = []
    context_words = []
    output_labels = []
    
    out_list = [1]
    for i in range(0,num_ns):
        out_list.append(0)
    
    for sentence in preprocess(vocab, counts, sentences, threshold=0.001):
        for target_index in range(0,len(sentence)):
            window_prob = random.randint(1, window+1)
            blahblah = int((window_prob-1)/2)
            for context_index in range(target_index-blahblah,target_index+blahblah+1):
                if(context_index < 0 or context_index >= len(sentence) or context_index == target_index):
                    continue
                else:
                    target_words.append([sentence[target_index]])
                    temp_list = []
                    #print(str(sentence[target_index]) + " " + str(sentence[context_index]))
                    #positive sampling
                    temp_list.append(sentence[context_index])
                    ns_samples = random.choices(range(0,len(counts)), [x**ns_exponent for x in counts], k=num_ns)
                    #nstemp_list = random.sample( vocab.items(), num_ns )
                    for i in ns_samples:
                        temp_list.append(i)
                    context_words.append(temp_list)
                    output_labels.append(out_list)
                    if(len(target_words) == batch_size):
                        yield target_words, context_words, output_labels
                        target_words = []
                        context_words = []
                        output_labels = []
        #print(target_words)
        #print(context_words)
        #print(output_labels)
    #yield torch.zeros(batch_size).long(), torch.zeros(batch_size, 1 + num_ns).long()

Your code should comply with the following specification:

**training_examples** (*vocab*, *counts*, *sentences*, *window* = 5, *num_ns* = 5, *batch_size* = 524,288, *ns_exponent*=0.75)

> Reads from an iterable of *sentences* (lists of string tokens), preprocesses them using the function implemented in Problem&nbsp;2, and then yields pairs of input batches for gradient-based training, represented as described above. Each batch contains *batch_size* positive examples. The parameter *window* specifies the maximal distance between a target word and a context word in a positive example; the actual window size around any given target word is sampled uniformly at random. The parameter *num_ns* specifies the number of negative samples per positive sample. The parameter *ns_exponent* specifies the exponent in the negative sampling (called $\alpha$ above).

### 🤞 Test your code

To test your code, compare the total number of positive samples (across all batches) to the total number of tokens in the (un-preprocessed) minimal dataset. The ratio between these two values should be ca. 2.64. If you can spare the time, you can make the same comparison on the full dataset; here, the expected ratio is 3.25. As before, the numbers may vary slightly because of randomness, so you may want to run the comparison more than once.

In [75]:
# TODO: Test your code here
import random
#11638
mini_dataset = SimpleWikiDataset(max_sentences=11638)
#full_dataset = SimpleWikiDataset()
vocab, counts = make_vocab_and_counts (mini_dataset, min_count = 5)
print (len(vocab))
print (len(counts))
#print (counts[0])
print (sum(counts))
#print (counts)
#print (vocab)
#mini_dataset_preprocess = preprocess(vocab, counts, mini_dataset, threshold=0.001)
#print(mini_dataset_preprocess)
#vocab_p, counts_p = make_vocab_and_counts (mini_dataset_preprocess, min_count = 5)
#def training_examples(vocab, counts, sentences, window=5, num_ns=5, batch_size=1<<19, ns_exponent=0.75):
training_examples(vocab, counts, mini_dataset, 3, 1, 0, 0.75)

3231
12437
155818


<generator object training_examples at 0x7ff9201149d0>

## Problem 4: Implement the model

Now it is time to implement the skip-gram model as such. The cell below contains skeleton code for this. As you will recall from Lecture&nbsp;1.4, the core of the implementation is formed by two embedding layers: one for the target word representations, and one for the context word representations. Your task is to implement the missing `forward()` method.

In [76]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SGNSModel(nn.Module):
    
    def __init__(self, vocab, embedding_dim):
        super().__init__()
        self.vocab = vocab
        self.w = nn.Embedding(len(vocab), embedding_dim)
        self.c = nn.Embedding(len(vocab), embedding_dim)
    
    def forward(self, w, c):
        # TODO: Replace the next line with your own code
        w = torch.as_tensor(w)
        c = torch.as_tensor(c)
        t = self.w(w)
        k = self.c(c)
        x = torch.mul(t, k)
        return torch.sum(x ,dim=1) 
        #return torch.zeros_like(c, dtype=torch.float, requires_grad=True)

Your implementation of the `forward()` method should comply with the following specification:

**forward** (*self*, *w*, *c*)

> The input to this methods is a tensor *w* with target words of shape $[B]$ and a tensor *c* with context words of shape $[B, 1+k]$, where $B$ is the batch size and $k$ is the number of negative samples. The two tensors are structured as explained for Problem&nbsp;3. The output of the method is a tensor $D$ of shape $[B, k+1]$ where entry $D_{ij}$ is the dot product between the embedding vector for the $i$th target word and the embedding vector for the context word in row $i$, column $j$.

**💡 Hint:** To compute a dot product $x^\top y$, you can first compute the Hadamard product $z = x \odot y$ and then sum up the elements of $z$.

### 🤞 Test your code

Test your code by creating an instance of the model, and check that `forward` returns the expected result on random input tensors *w* and *c*. To help you, the following function will return a random example from the first 100 examples produced by `training_examples`.

In [82]:
# TODO: Test your code here
model = SGNSModel(vocab, 5)
t, c, l = random_example(vocab, counts, mini_dataset)
#print(len(t))
#print(t)
#print(len(c))
#print(c)
#for i in range(len(t)):
x = model.forward(t, c)
print(x)

tensor([[ 0.1721, -0.0842,  0.7075, -2.3163,  0.0232],
        [-0.2913,  0.6251,  1.7461, -0.0982,  1.2497],
        [-0.0391,  0.8775,  0.3141, -0.2841, -0.9317],
        [ 1.5838, -1.3281, -0.3017,  1.3644,  0.8496],
        [ 1.2530, -0.2365,  0.1726, -1.1046,  0.7329]], grad_fn=<SumBackward1>)


In [77]:
import numpy as np

def random_example(vocab, counts, sentences):
    skip = np.random.randint(100)
    for i, example in enumerate(training_examples(vocab, counts, sentences, num_ns=1, batch_size=5)):
        if i >= skip:
            break
    return example

## Problem 5: Train the model

Once you have a working model, it is time to train it. The training loop for the skip-gram model will be very similar to the prototypical training loop that you saw in Lecture&nbsp;0.6, with two things to note:

First, instead of categorical cross entropy, you will use binary cross entropy. Just like the standard implementation of the softmax classifier, the skip-gram model does not include a final non-linearity, so you should use [`binary_cross_entropy_with_logits()`](https://pytorch.org/docs/1.9.1/generated/torch.nn.functional.binary_cross_entropy_with_logits.html).

The second thing to note is that you will have to create the tensor with the output labels, as explained already in Problem&nbsp;3. This should be a matrix of size $[B, 1+k]$ whose first column contains $1$s and whose remaining columns contains $0$s.

Here is skeleton code for the training loop, including default values for the most important hyperparameters:

In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim

def train(sentences, embedding_dim=50, window=5, num_ns=5, batch_size=1<<19, n_epochs=1, lr=1e-1):
    # Create the vocabulary and the counts
    vocab, counts = make_vocab_and_counts(sentences)
    
    # Initialize the model
    model = SGNSModel(vocab, embedding_dim)
    
    # Initialize the optimizer. Here we use Adam rather than plain SGD
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # TODO: Add your code here
    
    return model

To show you how `train` is meant to be used, the code in the next cell trains a model on the minimal dataset.

In [ ]:
model = train(mini_dataset, n_epochs=1)

### 🤞 Test your code

Test your implementation of the training loop by training a model on the minimal dataset. This should only take a few seconds. You will not get useful word vectors, but you will be able to see whether your code runs without errors.

Once you have passed this test, you can train a model on the full dataset. Print the loss to check that the model is actually learning; if the loss is not decreasing, try to find the problem before wasting time (and energy) on useless training.

Training on the full dataset will take some time – on a CPU, you should expect 10–40 minutes per epoch, depending on hardware. To give you some guidance: The total number of positive examples is approximately 58M, and the batch size is chosen so that each batch contains roughly 10% of these examples. To speed things up, you can train using a GPU; our reference implementation runs in less than 2 minutes per epoch on [Colab](http://colab.research.google.com).

In [ ]:
# TODO: Train your model on the full dataset here

## Problem 6: Analyse the embeddings (reflection)

Now that you have a trained model, you will probably be curious to see what it has learned. You can inspect your embeddings using the [Embedding Projector](http://projector.tensorflow.org). To that end, click on the ‘Load’ button, which will open up a dialogue with instructions for how to upload embeddings from your computer.

You will need to upload two tab-separated files. To create them, you can use the following code:

In [ ]:
def save_model(model):
    # Extract the embedding vectors as a NumPy array
    embeddings = model.w.weight.detach().numpy()
    
    # Create the word–vector pairs
    items = sorted((i, w) for w, i in model.vocab.items())
    items = [(w, e) for (i, w), e in zip(items, embeddings)]
    
    # Write the embeddings and the word labels to files
    with open('vectors.tsv', 'wt') as fp1, open('metadata.tsv', 'wt') as fp2:
        for w, e in items:
            print('\t'.join('{:.5f}'.format(x) for x in e), file=fp1)
            print(w, file=fp2)

Take some time to explore the embedding space. In particular, inspect the local neighbourhoods of words that you are curious about, say the 10 closest neighbours. Document your exploration in a short reflection piece (ca. 150&nbsp;words). Respond to the following prompts:

* Which words did you try? Which results did you get? Did you do anything else than inspecting local neighbourhoods?
* Based on what you know about word embeddings, did you expect your results? How do you explain them?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

*TODO: Enter your text here*

👍 Well done!